# Random prime number generation
(using only a random bits generator and a probabilistic primality test)

In [1]:
#libraries we use in this file
import random,sys #random for random bits generator
import sympy as sp #sympy for some arithmetic computations (can be replaced by your own implementation!)

In [25]:
#Random k-bit number generator
def gennum(size,even=None):
    li=[random.choice([0,1]) for k in range(0,size-2)]
    if even=='even':
        li.append(0)
    elif even=='odd':
        li.append(1)
    else:
        li.append(random.choice([0,1]))
    num=li[0]
    for el in li[1:]:
        num<<=1
        num^=el
    return num

In [6]:
#Primality test
def GCD(a,b):
    while b:   
        a, b = b, a % b
    return abs(a)

def IsComposite(a,n):
    d=GCD(a,n)
    if (d==1):
        a0=pow(a,n-1,n)
        if (a0 == 1):
            return False #"Not conclusive"
        else:
            return True
    else:
        return True
    
def IsCompositeTest(x,N):
    for _ in range(0,N):
        a=random.randrange(2,x-1)
        d=GCD(a,x)
        if d!=1:
            return True
        elif IsComposite(a,x):
            return True
    return False

In [17]:
IsCompositeTest(113423432423421,100)

True

In [26]:
#Pseudo-prime number generator
def GenPseudoPrime(N,size):
    num=gennum(size,'odd')
    while IsCompositeTest(num,size):
        num=gennum(size,'odd')
    return num

In [ ]:
#Example

In [24]:
GenPseudoPrime(100,500)

1360236714310743036105081483215152151545989264597680428123055686994492364545559977608559980856774249312586631295801241096963238822448037503375062517473

In [21]:
sp.factorint(GenPseudoPrime(100,400))

{1953615246307953479300330063308762230826441657462094743380695508775950985718611397538199693322960986150159403788563475373: 1}

# RSA algorithm

## Private and public key generation

1. Generate two large random (and distinct) primes $p$ and $q$ of the same size.

In [27]:
def GenTwoPrimes(size,N):
    while True:
        p=GenPseudoPrime(N,size)
        q=GenPseudoPrime(N,size)
        if p!=q:
            return p,q

In [29]:
GenTwoPrimes(10,5)

(173, 373)

2. Compute $n=pq$ and $\phi=\phi(n)=(p-1)*(q-1)$

Remark: discuss why the number $lcm(p-1,q-1)$ can be used instead. When it differs from $\phi$.

In [30]:
def ProdModulus(size,N):
    p,q=GenTwoPrimes(size,N)
    n=p*q
    phin=(p-1)*(q-1)
    return n,phin

In [31]:
#example
ProdModulus(20,10)

(2684634413, 2684491740)

In [ ]:
#n=p*q
#phi-n-1=-(p+q)
#(x-p)*(x-q) == x**2-(p+q)*x+p*q == x**2+(phi-n-1)*x+n

In [34]:
x=sp.symbols('x')
phi=2684491740
n=2684634413
sp.solve(x**2+(phi-n-1)*x+n,x)

[22303, 120371]

In [35]:
22303*120371
#Do not reveal number phi

2684634413

3. Choose a random integer $e$ in the range $1<e<\phi$ such that $gcd(e,\phi)=1$.

In [40]:
def GCD(a,b):
    while b:   
        a, b = b, a % b
    return abs(a)
def RandomE(phin):
    size=phin.bit_length()
    sizen=random.randrange(2,size)
    while True:
        num=gennum(sizen)
        if GCD(num,phin)==1 and num>1:
            return num

In [41]:
#example
_,phin=ProdModulus(20,10)
RandomE(phin)

56339

4. Use the extended Euclidean algorithm to compute the unique integer $d$, $1<d<\phi$, such that $ed\equiv 1(\textrm{mod }\phi)$

In [43]:
def ExtendedGCD(a,b):
    r,r1=a,b
    s,s1=1,0 #s*a+t*b == a
    t,t1=0,1 #s1*a+t1*b == b
    while not(r1==0):
        q,r2=r//r1,r % r1
        r,s,t,r1,s1,t1=r1,s1,t1,r2,s-s1*q,t-t1*q
    d=r
    return d,s,t #s*a+t*b=d, d=GCD(a,b)

def ComputeD(e,phi):
    _,inv,_=ExtendedGCD(e,phi)
    return inv%phi

In [45]:
#example
_,phin=ProdModulus(200,110)
e=RandomE(phin)
d=ComputeD(e,phin)
print(bool((e*d-1)%phin==0)) #sanity check
print(phin)
print(e)
print(d)

True
128497686453572616304094372429668827259662801480626294717309774850736790475254664321147787129198622229502318816001144844
546081154169387807342449270317018301968250332952159872118506035260939878011645
14004926958874383052088759194238999709654386960074449894349322545055865971608279965706014908118503483987109354561089561


5. Alice's public key is $(n,e)$, private key is $d$.

In [46]:
def PrivatePublicKeyRSA(size,N=200):
    n,phin=ProdModulus(size,N)
    e=RandomE(phin)
    d=ComputeD(e,phin)
    return d,(n,e)

In [47]:
#example of key generation with two primes of size 300 bits
AlicePriv,AlicePub=PrivatePublicKeyRSA(300) #default pseudoprimality check at 200 witness choices

In [48]:
AlicePriv

197554619374451336632417877822140819625957999871551099253742431111893570074114939710984283571151883694297146465575029721786804053289650949472537675722334463204465955731228733878273

In [49]:
AlicePub

(280162496357024280138077426027361239815390446861476863167822579201508023332841609897508196034609598100036189660526837875734936053101632366863397944015718778712656384089945462331713,
 33348290189216285550711884374147204951263520202492748756309044024900351337531037807759719725707304865964018333475777)

## RSA public-key encryption

1. Bob encrypts a message $m$ for Alice using her public key $(n,e)$

We assume that the message $m$ is an integer in the interval $0\leq m\leq n-1$.

In [50]:
def EncryptRSA(pub,m):
    n,e=pub
    c=pow(m,e,n) #fast modular exponentiation
    return c

In [51]:
#example
_,AlicePub=PrivatePublicKeyRSA(300) 
m=123456789
c=EncryptRSA(AlicePub,m)
c

47995494908001224731212277472819204556838016550452016517503919510065505227858409332498253645928130557216039313957183880025748947396787820722823027725591137859325357797854177861616

In [60]:
#self-encryption example
p=5
q=7
n=p*q
e=3
m=6
c=EncryptRSA((n,e),m)
c

6

2. Alice decrypts the message $c$ using her private key $d$ and part of the public key $n$

In [61]:
def DecryptRSA(priv,pub,c):
    d=priv
    n,_=pub
    m=pow(c,d,n)
    return m

In [62]:
#example
AlicePriv,AlicePub=PrivatePublicKeyRSA(300) 
m=123456789
c=EncryptRSA(AlicePub,m)
bool(DecryptRSA(AlicePriv,AlicePub,c)==m)

True

In [63]:
size=500
%timeit PrivatePublicKeyRSA(size)
AlicePriv,AlicePub=PrivatePublicKeyRSA(size)
m=12345678910123456789101234567891012345678910
%timeit EncryptRSA(AlicePub,m)
c=EncryptRSA(AlicePub,m)
%timeit DecryptRSA(AlicePriv,AlicePub,c)
bool(DecryptRSA(AlicePriv,AlicePub,c)==m)

13 s ± 5.55 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
23.5 ms ± 6.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
27.9 ms ± 9.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


True

Exercise: Implement RSA decryption using Chinese Remainder Theorem. Compare timings.

# Attacks on RSA

1. We implement an attack which shows that once the adversary knows $d$, she can implement an attack which factors $n$ efficiently.

In [64]:
nn=14205142842144491469901035779943007321473952670460614909740188710462796861921791780746014298824348546889748863603913825380912304112461129061114480661500416910991853573649055897001583708234998530660447745535711467407798340361335928981312718926721467943464464347521000503179497153112764130114342341251457556854374337702225661788558784747007799183865452550277915792606190524979919835785502848268656744723582283945123371679980696891117277548547543492116459573915049465031893477375432302554045103150951955486083526016584926750095118984741954481489582827589374811855794969993254570253121737541317841105374871
dd=9738454175598488918517912045396815318351885031131011603301149540233201870415928124228184903947308481461717153640402767289853198952704967449300122329014740408508653613839688094250923162490670540988214688775753190900423588412005697560323304500348114898045236656807283167901253083798426709790746938525240264995502098847606530252043043212677911465343705421183831116604350283789270965024124861992541018116786274867535581082248878546385006259988838129620903989258127062367035340066868353921340378027331177496332241490297041686454303452932424111634076797215417394272455217584601075851777273706083879476230809
ee=2219702669760051625529760071259189046161364151701596790770763259600544290997125107128138578832480323854037838605599695123440903054424577956799678397891626783444723950147784407335462559143107157658471735164714153971357443698994082727673072343180069044835094856719244582969485137575845153825021391095268519544748057926663150576101990156077844973202826679622719216615756960610764785110408304311098865781072786879379296360025429207038042833064515876868608188436266546466015175298619766069707237580766787423687287858279125035537409323009740621048068813783768774814593993312720811077575752373741693972477513

In [65]:
GCD(5,ee*dd-1)

1

In [66]:
a=5
pow(a,ee*dd-1,nn)

1

In [67]:
def FactorN(a,N):
    s=0
    while a%N==0:
        a//=N
        s+=1
    return a,s

In [68]:
t,s=FactorN(ee*dd-1,2)

In [69]:
def TestRes(a,i,t,n):
    val=pow(a,2**(i-1)*t,n)
    cond1=not(val in [n-1,1])
    cond2=(pow(val,2,n)==1)
    return cond1 and cond2

In [70]:
[i for i in range(1,s+1) if TestRes(5,i,t,nn)]

[4]

In [71]:
p=GCD(pow(a,2**(4-1)*t,nn)-1,nn)
p

9588619046366838044801494053253481940617720553952621475995094046755114382462934062374744942467354034938065773299505136257219654734188396834557125565877492393644737116271016592052764192115682498712093244477233001835608690898970597314134231687516574207045955396893074444391250211321989449626922987925383

In [72]:
q=GCD(pow(a,2**(4-1)*t,nn)+1,nn)
q

1481458672354583818459092744470727113302331222529818045554004506818880990463267536139772114225085488389179096710354485475436883604179235185470105001402382279412556095502878969668106727661199802579922297746837666119028151509739065244008176124005425474201446524659942015313448803439947198799383945677937

In [75]:
print(p*q==nn)
print(p>1)
print(q>1)
print(p<nn)
print(q<nn)



True
True
True
True
True


Exercise: implement it as a general procedure to factor the modulus $n$ of RSA provided that $d$ is explicitely given.

2. For practical purposes one can choose an exponent $e$ to be very small. The smalles such choice is often $e=3$.

Warning: we should assume that neither $p-1$ nor $q-1$ is divisible by $3$.

Suppose three entities generated public keys $(n_1,3)$, $(n_2,3)$ and $(n_3,3)$. Bob has generates three messages $c_i=m^3(\textrm{ mod }n_i)$.

Using a CRT theorem we can find a unique $x$ such that $0\leq x<n_1 n_2 n_3$ and $x\equiv c_i (\textrm{ mod }n_i)$. 

Since $m^3<n_1 n_2 n_3$ then it follows from the uniqueness that $x=m^3$ !

So now simply we need to compute a cubic root of $x$ to find the message $m$.

(We assume that pairwise the $n_i$ are coprime).

In [ ]:
def MultInverse(a,n):
    d,inv,_=ExtendedGCD(a,n)
    if d==1:
        if n==1:
            return 1 #for compatibility
        return inv%n
    else:
        raise NameError('Numbers '+str(a)+' and '+str(n)+' are not coprime.')
def GaussAlgorithm(clist,nlist):
    n=sp.prod(nlist)
    x=0
    for i in range(0,len(clist)):
        ni=n//nlist[i]
        mi=MultInverse(ni,nlist[i])
        x+=clist[i]*ni*mi
    return x%n

In [ ]:
def PrivatePublicKeyRSAForceE(size,e,N=200):
    n,phin=ProdModulus(size,N)
    d=ComputeD(e,phin)
    return d,(n,e)

In [ ]:
pr1,pub1=PrivatePublicKeyRSAForceE(300,3)
pr2,pub2=PrivatePublicKeyRSAForceE(300,3)
pr3,pub3=PrivatePublicKeyRSAForceE(300,3)

In [ ]:
m=122333444455555666666

In [ ]:
c1=EncryptRSA(pub1,m)
c2=EncryptRSA(pub2,m)
c3=EncryptRSA(pub3,m)

In [ ]:
n1=pub1[0]
n2=pub2[0]
n3=pub3[0]

In [ ]:
gg=(GaussAlgorithm([c1,c2,c3],[n1,n2,n3]))
sp.root(gg,3)

Exercise: how the n-rooth algorithm can be designed, knowing that the input is a perfect power?

3. Suppose that $\phi(n)$ was computed. Knowing $(n,e)$ the public key, can you find the private key $d$?

In [ ]:
p,q,n,phi,x=sp.symbols('p,q,n,phi,x')
eq1=sp.Eq(p*q,n) #p*q==n
eq2=sp.Eq((p-1)*(q-1),phi) #(p-1)*(q-1)==phi

In [ ]:
((eq1.lhs-eq2.lhs).expand(),(eq1.rhs-eq2.rhs).expand())

In [ ]:
#(x-p)*(x-q)==x**2-(p+q)*x+p*q == x**2-(n-phi+1)*x+n

In [ ]:
sp.solve(x**2-(n-phi+1)*x+n,x)

In [ ]:
def PrivatePublicKeyRSALeak(size,N=200):
    n,phin=ProdModulus(size,N)
    e=RandomE(phin)
    d=ComputeD(e,phin)
    return d,(n,e),phin

In [ ]:
priv,pub,phi=PrivatePublicKeyRSALeak(300)

In [ ]:
def RevealPQ(n,phi):
    sq=sp.root(n**2 - 2*n*phi - 2*n + phi**2 - 2*phi + 1,2)
    p=(n - phi - sq+1)//2
    q=(n - phi + sq+1)//2
    return p,q

In [ ]:
p,q=RevealPQ(pub[0],phi)
p*q==pub[0]

4. Common modulus attack:
Suppose one generates pairs $(n,e_1)$ and $(n,e_2)$ with common modulus and coprime $e_1$ and $e_2$. Could you design an attack based on that information and two cryptograms $c_1,c_2$ which originate in the same message $m$.

Hint: notice that $RSA$ is multiplicative ,i.e. $c_1 c_2\equiv c(\textrm{ mod }n)$ where $c=m^{e_1 e_2}(\textrm{ mod n})$.

Use the extended Euclidean algorithm to find two integers $x,y$ such that $xe_1+ye_2=1$. 